In [25]:
# Skriptas rpeaks sekų, gautų iš .json failo ir EKG įrašo, panaudojant NEUROKIT2, sulyginimui
# Nuskaitomi įrašų failai .npy 

from xmlrpc.client import SYSTEM_ERROR
import pandas as pd
import numpy as np
import json, shutil, sys
import neurokit2 as nk
from pathlib import Path
from vertinimas_util import AnalyseHeartrate


def zive_read_file_1ch(filename):
    f = open(filename, "r")
    a = np.fromfile(f, dtype=np.dtype('>i4'))
    ADCmax=0x800000
    Vref=2.5
    b = (a - ADCmax/2)*2*Vref/ADCmax/3.5*1000
    ecg_signal = b - np.mean(b)
    return ecg_signal

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'
# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_2022_RUDUO_2'

#  Nuoroda į aplanką su ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_selected')
rec_dir_eval = Path(db_path, 'records_evaluated')

# Variantas: visi duomenys
# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
# file_path = Path(rec_dir,'info_create_z.json')
# with open(file_path) as json_file:
#     info_create = json.load(json_file)
# SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas


print("Skriptas rpeaks sulyginimui. Lyginamas rpeaks masyvas, suformuotas iš json failo,")
print("nuskaityto iš Zive frontendo ir rpeaks masyvas, gautas iš EKG signalo naudojant")
print("Neurokit2 funkciją ecg_peaks. EKG signalas (įrašas) irgi gautas iš Zive frontendo.")
print("Abu rpeaks masyvai įrašomi į json vizualiniam sulyginimui su http://www.jsondiff.com/")
print("rpeaks iš json failo varde pridėta '-js', rpeaks iš signalo - pridėta '-sg'")

# Įrašai, kur rpeak nesutapimas tik 1
list = ["1636423.936"] # 0
list = ['1636490.526'] # 0
list = ['1636495.646'] # 0
list = ['1636499.394'] # 0
list = ['1632133.116'] # 0
list = ['1636563.972'] # 0
list = ['1636593.000'] # 0
list = ['1631141.137'] # 0
list = ['1631129.850'] # 0
list = ['1631082.735'] # 0
list = ['1631082.118'] # 0
list = ['1631081.543'] # Nuolat lenda raumenų triukšmai, yra karts nuo karto nedideli izolinijos šuoliai
list = ['1637166.430'] # 1, Ties 13000 yra atkarpa su stipriais raumenų (ar kitokio pobūdžio triukšmais). Bet izolinija stabili.
list = ['1637178.685'] # 1
list = ['1633540.147']
list = ['1633958.174']
list = ['1630717.159']
list = ['1632123.567'] # 0
list = ['1632728.750'] # 10 Rpeaks, kurių Neurokit nesukuria, mediko sudėta, visi yra V, ML irgi pagauna
list = ['1643061.340'] # 10 rpeaks, ten, kur jų nėra, mediko panaikinta

list = ['1636451.235'] # 1, Važinėjanti izolinija, pastebimi raumenų triukšmai
list = ['1631066.094'] # 0, Yra triukšmelis ties 9800. Nuėmiau rpeaks ties 49000 ir 50800, neaiškumas ties 75400
list = ['1637584.237'] # Pavirpėjimas prie 109200
list = ['1631047.940'] # Kartais išlenda pastebimi raumenų triukšmai
list = ['1631039.807'] # 0
list = ['1637584.866'] # 0  
list = ['1637585.491'] # 0
list = ['1631027.284'] # 0
list = ['1637619.150'] # 1, Labai važinėjanti izolinija, kartais staigiai, raumenų triukšmų palyginus nedaug 
list = ['1631002.473'] # 1, Iki vidurio kokybė gera, bet nuo vidurio yra tarpų su izolinijos šokinėjimu
# ir raumenų triukšmais. Gerai ML atpažista V

list = ['1633444.221'] # 1, Yra izolinijos staigių pavaikščiojimų, raumenų triukšmų išlindimų. Dažnai N arba S ML atpažista, kaip V.
list = ['1630999.344'] # 0, Yra tik vienoje vietoje staigus izolinijos šoktelėjimas su triukšmų išlindimu ties 119000, ML V gaudo neblogai, yra dubletų, tripletų
list = ['1637628.331'] # 0, Nemažai S, ML juos neblogai gaudo

list = ['1637628.402'] # 0, Iššoka triukšmai tik ties 48000.
list = ['1637631.801'] # 0
list = ['1637189.804'] # 0, Reiktų parodyti gydytojui. išmestas rpeak 72858, nors man atrodo, kad jis ten yra, ir ten ekstrasistolė. Yra ir daugiau vietų, atrodo, kaip ekstrasistolės, tačiau nepažymėta. Reiktų parodyti gydytojui.
list = ['1636414.518'] # 0, Sutriukšmauja tik ties 81200 ir 119200
list = ['1634736.586'] # 1, Nestabili izolinija, kartais išlenda triukšmai
list = ['1632923.661'] # 1, Labai ilgas, > 4000. Praleistas pikas, gydytojas pažymėjo. Kokybė nėra aiški
list = ['1630844.567'] # 2

# 2023 01 13 trūko 1:30
list = ['1633443.596'] # 1, Gera kokybė suprastėja nuo 93000, daug S, ML nemažai S ir N priskiria V
list = ['1633434.823'] # 1, Buvo praleistas rpeaks, gydytojas įdėjo. Yra staigių izolinijos šokinėjimų.
list = ['1633434.198'] # 0
list = ['1633427.309'] # 2, Ties 30400 prasideda izolinijos  šokinėjimas
list = ['1633548.426'] # 0, Yra pašokinėjimų nuo 30000, bet nedaug
list = ['1633408.989'] # 0
list = ['1633408.480'] # 2, Yra gana didelės amplitudės raumenų triukšmų
list = ['1633396.439'] # 2
list = ['1634648.967'] # 0
list = ['1633395.185'] # 1
list = ['1633387.028'] # 1

# 2023 01 18 trūko 1:30
# Šiuose 4 failuose rpeaks visi neteisingi, dar sykį leidžiant Neurokit - rpeaks
# atstatomi į teisingas vietas. json sutvarkomas su skriptu 1_rpeaks_json_koregavimas.ipynb
list = ['1637624.073'] # Nesutampančių yra 1481, taisomas specialiomis priemonėmis
list = ['1637624.073_new'] # 1,  rpeaks pataisyti, Kokybė arti 0, 
# tačiau yra keleta fragmentų su ryškiais raumenų triukšmais. Rpeaks pataisyti su spec. skriptu
# list = ['1637621.569'] # nesutampančių 1354,  taisomas specialiomis priemonėmis
list = ['1637621.569_new'] # 0, pataisytas
# list = ['1637630.258'] # nesutampančių 1327,  taisomas specialiomis priemonėmis
list = ['1637630.258_new'] # 1, pataisytas
# list = ['1638807.019'] # nesutampančių 1192,  taisomas specialiomis priemonėmis
list = ['1638807.019_new'] # 0,  pataisytas

# list = ['1638818.942'] #0,  nesutampančių 93,  38695 rpeak pakoreguotas ne ten (atstačiau su Zive), 
# # likusieji visi surasti ties nedideliu piku tarp gretimų R, anotatoriaus panaikintų rpikų: 92, pridėtų: 0  
# Labai įdomus įrašas, reiktų gydytojo paaiškinimo: yra daug dvigubai didesnių RR intervalų.
list = ['1638818.315'] # 0,  nesutampančių 71, visi surasti ties nedideliu piku tarp gretimų R,
# anotatoriaus panaikintų rpikų: 71,, pridėtų: 0     
# Labai įdomus įrašas, reiktų gydytojo paaiškinimo: yra daug dvigubai didesnių RR intervalų.
list = ['1642628.030'] # 0,  nesutampančių 53, anotatoriaus panaikintų rpikų: 53, pridėtų: 0

# 2023 01 20 trūko 2:00
list = ['1642627.410'] # 0,  nesutampančių 41, anotatoriaus panaikintų rpikų: 41, pridėtų: 0    
list = ['1638801.784'] # 2 ,  nesutampančių 40, anotatoriaus panaikintų rpikų: 22, pridėtų: 18
list = ['1637623.546'] # 0 ,  nesutampančių 37, anotatoriaus panaikintų rpikų: 37, pridėtų: 0
#  - nesutampa N, ir po analizės - U
list = ['1637622.288'] # 1 ,  nesutampančių 33, anotatoriaus panaikintų rpikų: 32, pridėtų: 1,
#  uždėta ne vietoje - pataisiau rankomis. Kokybė arti 0, tačiau pradžioje yra didelis izolinijos lakstymas
list = ['1637624.172'] # 0 ,  nesutampančių 32, anotatoriaus panaikintų rpikų: 32, pridėtų: 0
list = ['1637624.799'] # 0 ,  nesutampančių 30, anotatoriaus panaikintų rpikų: 30, pridėtų: 0
list = ['1636596.137'] # 0 ,  nesutampančių 29, anotatoriaus panaikintų rpikų: 29, pridėtų: 0
list = ['1637622.914'] # 0 ,  nesutampančių 28, anotatoriaus panaikintų rpikų: 24, pridėtų: 4

# 2023 01 23 trūko X:00
list = ['1632252.683'] # 0 ,  nesutampančių 25, anotatoriaus panaikintų rpikų: 25, pridėtų: 0
# Trumpas triukšmelis ties 115750
list = ['1638817.688'] # 0 ,  nesutampančių 25, anotatoriaus panaikintų rpikų: 25, pridėtų: 0
# Yra nemažai pauzių
list = ['1640590.013'] # 2 ,  nesutampančių 24, anotatoriaus panaikintų rpikų: 22, pridėtų: 2
list = ['1643070.532'] # 1 ,  nesutampančių 22, anotatoriaus panaikintų rpikų: 20, pridėtų: 2
# Gana bjauri vieta ties 14300
list = ['1636596.765'] # 0 ,  nesutampančių 22, anotatoriaus panaikintų rpikų: 21, pridėtų: 1
list = ['1632731.258'] # 0 ,  nesutampančių 22, anotatoriaus panaikintų rpikų: 21, pridėtų: 1
list = ['1632729.377'] # 0 ,  nesutampančių 19, anotatoriaus panaikintų rpikų: 0, pridėtų: 19
# Kartais išlenda aiškiai matomi raumenų triukšmai, daug nesurastų rpeaks ties V, sužymėti ranka
list = ['1638803.024'] # 2 ,  nesutampančių 19, anotatoriaus panaikintų rpikų: 11, pridėtų: 8
list = ['1638799.296'] # 2 ,  nesutampančių 19, anotatoriaus panaikintų rpikų: 13, pridėtų: 6
list = ['1638799.296'] # 2 ,  nesutampančių 19, anotatoriaus panaikintų rpikų: 13, pridėtų: 6
list = ['1643065.093'] # 0 ,  nesutampančių 18, anotatoriaus panaikintų rpikų: 18, pridėtų: 0
list = ['1642630.518'] # 0 ,  nesutampančių 18, anotatoriaus panaikintų rpikų: 18, pridėtų: 0
list = ['1637626.680'] # 0 ,  nesutampančių 17, anotatoriaus panaikintų rpikų: 16, pridėtų: 1
list = ['1632730.004'] # 0 ,  nesutampančių 17, anotatoriaus panaikintų rpikų: 0, pridėtų: 17
# Kartais išlenda nedidelis raumenų triukšmas, daug praleista rpeaks ties V, sužymėti ranka
list = ['1646711.162'] # 1 ,  nesutampančių 17, anotatoriaus panaikintų rpikų: 17, pridėtų: 0
# Labai šokinėjanti izolinija
list = ['1642635.482'] # 0 ,  nesutampančių 16, anotatoriaus panaikintų rpikų: 16, pridėtų: 0
# Gana stabilus įrašas, tačiau nuolat matosi nedidelis raumenų triukšmas

# 2023 01 26 trūko X:00
list = ['1643078.826'] # 1,  nesutampančių 16, anotatoriaus panaikintų rpikų: 16, pridėtų: 0
# Didžioji dalies kokybė gera, tik nedideli raumenų triukšmai, tačiau nuo 106000 prasideda 
# kontaktiniai ir raumenų triukšmai, izolinijos nestabilumas.
list = ['1637625.426'] # 0,  nesutampančių 15, anotatoriaus panaikintų rpikų: 14, pridėtų: 1
list = ['1632730.631'] # 0,  nesutampančių 15, anotatoriaus panaikintų rpikų: 0, pridėtų: 15
# Neurokit blogai gaudo rpikus ties V
list = ['1638806.132'] # 2,  nesutampančių 15, anotatoriaus panaikintų rpikų: 9, pridėtų: 6
# Nuo 80600 prasidėjo staigūs izolinijos šuoliai ir neaiškūs triukšmai
list = ['1631037.236'] # 2,  nesutampančių 13, anotatoriaus panaikintų rpikų: 5, pridėtų: 8
list = ['1638829.105'] # 1,  nesutampančių 13, anotatoriaus panaikintų rpikų: 10, pridėtų: 3
# Triukšmų nedaug, tik išlenda pačioje pabaigoje, tačiau labai vaikščioja izolinija
list = ['1643073.813'] # 1,  nesutampančių 13, anotatoriaus panaikintų rpikų: 13, pridėtų: 0
# Arčiau 0, bet yra keleta pašokinėjančių trumpų atkarpų.
list = ['1642633.618'] # 0,  nesutampančių 12, anotatoriaus panaikintų rpikų: 12, pridėtų: 0
# Kaip ir daug kur, neurokit suranda rpeaks, kur vizualiai visai nepanašu ("lygioje vietoje")
list = ['1643064.468'] # 0,  nesutampančių 11, anotatoriaus panaikintų rpikų: 11, pridėtų: 0
list = ['1642632.378'] # 0,  nesutampančių 11, anotatoriaus panaikintų rpikų: 11, pridėtų: 0
# Nėra labai švarus, tačiau priskyriau 0


# 2023 02 10 truko X:00
list = ['1638830.348'] # 0,  nesutampančių 10, anotatoriaus panaikintų rpikų: 10, pridėtų: 0
# Triukšmingas ir šokinėjanti izolinija, tačiau ir triukšmai nedideli ir šokinėjimo amplitudė
#  maža ir ne staigiai. Neurokit randa pikus ties beveik nematomais pikais
list = ['1625405.103'] # 1,  nesutampančių 10, anotatoriaus panaikintų rpikų: 3, pridėtų: 7
# izolinija vietomis  pašokinėja, nemažai izolinijos bangavimų, yra ir raumenų triukšmų
list = ['1642634.862'] # 0,  nesutampančių 10, anotatoriaus panaikintų rpikų: 10, pridėtų: 0
# Vidutinio intensyvumo raumenų triukšmas nuo 38600, 3-5 pikai 

# 2023 04 24 truko X:00
list = ['1642632.998'] # 0,  nesutampančių 10, anotatoriaus panaikintų rpikų: 10, pridėtų: 0
# Nedideli EMG ir BW
# 2023 04 26 truko X:00
list = ['1643060.093'] # 0, labai švarus,   nesutampančių 9, anotatoriaus panaikintų rpikų: 9, pridėtų: 0
list = ['1642631.138'] # 0, labai švarus,   nesutampančių 9, anotatoriaus panaikintų rpikų: 9, pridėtų: 0
# Beveik visus V priskiria S
list = ['1642629.274'] # 0, labai švarus,   nesutampančių 9, anotatoriaus panaikintų rpikų: 9, pridėtų: 0
# Beveik visus V priskiria S
list = ['1638796.192'] # 2, Daug EMA, ypač pirmoje pusėje,   nesutampančių 9, anotatoriaus panaikintų rpikų: 5, pridėtų: 4
list = ['1638802.404'] # 1, Pradžioje daug EMA,   nesutampančių 9, anotatoriaus panaikintų rpikų: 5, pridėtų: 4
list = ['1630974.317'] # 0,  nesutampančių 9, anotatoriaus panaikintų rpikų: 9, pridėtų: 0
list = ['1638831.587'] # 0,  nesutampančių 9, anotatoriaus panaikintų rpikų: 9, pridėtų: 0
# Nedideli EMG ir BW
list = ['1642626.785'] # 0,  nesutampančių 9, anotatoriaus panaikintų rpikų: 9, pridėtų: 0
# Nedideli EMG ir BW, ties 113400 trumpas EMA
list = ['1643064.901'] # 0,  nesutampančių 9, anotatoriaus panaikintų rpikų: 8, pridėtų: 1
# Nedideli EMG ir BW
list = ['1637626.053'] # 0,  nesutampančių 8, anotatoriaus panaikintų rpikų: 8, pridėtų: 0
# Labai nedideli BW ir EMG, keistos formos V
list = ['1643065.721'] # 0,  nesutampančių 8, anotatoriaus panaikintų rpikų: 8, pridėtų: 0
# Daug blogai atpažintų S
list = ['1638817.061'] # 0,  nesutampančių 8, anotatoriaus panaikintų rpikų: 8, pridėtų: 0
# 
list = ['1643056.964'] # 0,  nesutampančių 7, anotatoriaus panaikintų rpikų: 7, pridėtų: 0
# 
# 2023 04 27 truko X:00
list = ['1643063.844'] # 0,  nesutampančių 7, anotatoriaus panaikintų rpikų: 7, pridėtų: 0
# Švarus
list = ['1643058.841'] # 0,  nesutampančių 7, anotatoriaus panaikintų rpikų: 7, pridėtų: 0
# Labai švarus
list = ['1630998.090'] # 0,  nesutampančių 7, anotatoriaus panaikintų rpikų: 7, pridėtų: 0
# Nedidelis BW
list = ['1630763.516'] # 1,  nesutampančių 7, anotatoriaus panaikintų rpikų: 0, pridėtų: 7
# daug BW, yra šokinėjančių BW, nedideli EMG
list = ['1630757.924'] # 1,  nesutampančių 7, anotatoriaus panaikintų rpikų: 0, pridėtų: 7
# daug BW, yra šokinėjančių BW, nedideli EMG
list = ['1625408.182'] # 1,  nesutampančių 7, anotatoriaus panaikintų rpikų: 3, pridėtų: 4
# Triukšminga judant, yra daug BW ir intensyvių EMG
list = ['1630757.303'] # 1,  nesutampančių 7, anotatoriaus panaikintų rpikų: 0, pridėtų: 7
# Lyg tai pradžioje išlenda EMA
list = ['1638801.160'] # 0,  nesutampančių 7, anotatoriaus panaikintų rpikų: 4, pridėtų: 3
# Vietomis intensyvūs BW
# Po valandos, dar valandą
list = ['1636498.771'] # 0,  nesutampančių 7, anotatoriaus panaikintų rpikų: 6, pridėtų: 1
# Nuolatinis nedidelis BW, nedideli EMG
list = ['1638794.330'] # 1,  nesutampančių 7, anotatoriaus panaikintų rpikų: 5, pridėtų: 2
# Yra daug BW, taip pat ir staigių BW, kartais pasitaiko EMA
list = ['1631058.853'] # 0,  nesutampančių 7, anotatoriaus panaikintų rpikų: 4, pridėtų: 3
# Ryškus BW tik pačioje pradžioje, toliau kokybė gera
list = ['1643063.215'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 6, pridėtų: 0
# 
list = ['1643059.466'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 6, pridėtų: 0
# 
list = ['1630992.454'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 6, pridėtų: 0
# Keletoje vietų išlindo BW
list = ['1630771.589'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 0, pridėtų: 6
# Gana silpnas BW

# 2023 04 28 truko X:00
list = ['1630764.137'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 0, pridėtų: 6
# Ryškus BW, bet be staigių šuolių
list = ['1630770.348'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 0, pridėtų: 6
# Stebimas BW
list = ['1630991.830'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 6, pridėtų: 0
# 
list = ['1625403.874'] # 1,  nesutampančių 6, anotatoriaus panaikintų rpikų: 4, pridėtų: 2
# BW, kartais staigūs, yra nemažai vietų su stipriu EMG 
list = ['1630890.629'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 2, pridėtų: 4
# Skilvelinės tachikardijos epizodas.
list = ['1642626.165'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 6, pridėtų: 0
# BW nedidelis, bet keletoje vietų su šuoliais
list = ['1643076.319'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 6, pridėtų: 0
# Kokybė gera, tik keletoje vietų ryškesnis BW ir nedidelis EGM
list = ['1643078.200'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 6, pridėtų: 0
# Kokybė gera, tik keletoje vietų ryškesnis BW, dažnai pastebimas nedidelis EGM
list = ['1643661.403'] # 0,  nesutampančių 6, anotatoriaus panaikintų rpikų: 6, pridėtų: 0
# Išlenda visų rūšių triukšmai: BW, EMG, kartais ir EMA

# 2023 04 29 truko X:00
list = ['1643062.591'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# 
list = ['1642629.898'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# Stebimas BW, keletoje vietų išlenda EMG
list = ['1630998.715'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# Ryškus BW, bei AMG epizodas, bet abu - tik pradžioje
list = ['1638803.644'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 3, pridėtų: 2
# Yra intensyvių BW, netgi staigių, keletoje vietų pereinamčių į EMA, daugiausia pirmoje įrašo dalyje
list = ['1630972.436'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# BW, kartais ryškus ir staigus, yra vietų su EMG 
list = ['1630768.486'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
#  
list = ['1643069.278'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# BW ir EMG 
list = ['1637181.774'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# Nedidelis BW

list = ['1630759.170'] # 1,  nesutampančių 5, anotatoriaus panaikintų rpikų: 1, pridėtų: 4
# Intensyvus BW, dažnas nelabai intensyvus EMG 
list = ['1632728.123'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 0, pridėtų: 5
#  Nedidelis BW, vietomis pasirodo nedidelis EMG
list = ['1632726.237'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 0, pridėtų: 5
#  Vietomis pasirodo nedidelis EMG
list = ['1632724.979'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 1, pridėtų: 4
# Kokybė pablogėja apie 50000, pasirodo net EMA
list = ['1638830.967'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 4, pridėtų: 1
# BW
list = ['1636503.145'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 4, pridėtų: 1
# Tik ties 97500 pasirodo epizodas su BW ir EMG
list = ['1638811.413'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# Yra epizodų su BW ir EMG
list = ['1631727.631'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# Silpnas EMG per visą EKG, vietomis sustiprėja
list = ['1630795.799'] # 1,  nesutampančių 5, anotatoriaus panaikintų rpikų: 3, pridėtų: 2
# Yra epizodų su BW, ties 38000 - EMA
list = ['1637277.147'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# Išskyrus epizodus ties 36000, BW yra nedidelis
list = ['1645142.882'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 5, pridėtų: 0
# Tik pora staigių BW epizodų
list = ['1643662.627'] # 0,  nesutampančių 5, anotatoriaus panaikintų rpikų: 4, pridėtų: 1
# Nedidelis EMG per visą įrašą, nemažai BW, yra EMA epizodų

# 2023 04 30 truko X:00
list = ['1638910.575'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Vienas rimtesnis BW epizodas ties 26000
list = ['1637615.384'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Keletas BW epizodų su staigiais šuoliais
list = ['1637627.307'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 3, pridėtų: 1
# Nemažai BW, keletas BW epizodų su staigiais šuoliais ir padidėjusiu EMG
list = ['1643060.716'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Labai švarus
list = ['1630997.466'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0

# 2023 05 02
list = ['1633939.352'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 3, pridėtų: 1
# Nedidelis BW, vietomis pasirodo nedidelis EMG
list = ['1633935.589'] # 2,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Daug EMA epizodų
list = ['1625402.027'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 2, pridėtų: 2
# izolinija vaikšto nedaug, vietomis išlenda raumenų triušmai, 13 V, bet atpažino tik 5, 8 min.
list = ['1630762.894'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 0, pridėtų: 4
# Neintensyvus BW, silpnas AMG
list = ['1630762.273'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 0, pridėtų: 4
# Ryškus BW, bet be staigių šuolių, silpnas EMG

# 2023 05 04
list = ['1636498.144'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Nedidelis, bet nuolatinois BW, kartais pasirodo labai nedidelis EMG
list = ['1630810.103'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 0, pridėtų: 4
# Labai švarus
list = ['1632725.610'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 0, pridėtų: 4
# Kartais nedidelis BW ir EMG 
list = ['1630973.692'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Yra BW, antroje dalyje yra epizodų su didele amplitude, yra EMG epizodų
list = ['1640024.852'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Yra keletas epizodų su BW ir EMG
list = ['1643075.066'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Silpnas BW ir EMG
list = ['1636598.650'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Labai švarus, išskyrus vieną epizodą ties 117500
list = ['1631030.505'] # 0,  nesutampančių 4, anotatoriaus panaikintų rpikų: 2, pridėtų: 2
# BW nedidelis ir pastovus, yra pora epizodų su šuoliais
list = ['1644276.762'] # 1,  nesutampančių 4, anotatoriaus panaikintų rpikų: 4, pridėtų: 0
# Yra nemažai epizodų su EMA
list = ['1633036.499'] # 2,  nesutampančių 4, anotatoriaus panaikintų rpikų: 1, pridėtų: 3
# Nuo 65000 prasideda didelės amplitudės EMG ir BW

# 2023 05 07
list = ['1638913.675'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Gera kokybė, išskyrus epizodą apie 24000
list = ['1638195.470'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Karts nuo karto pasirodo nedidelis EMG
list = ['1643055.715'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Labai švarus
list = ['1638911.195'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# EMG ir BW epizodai ties 27500, 100000, šiaip įrašas gana švarus
list = ['1637621.030'] # 1,  nesutampančių 3, anotatoriaus panaikintų rpikų: 2, pridėtų: 1
# Daug BW, taip pat su šuoliais, kartais išlenda EMG
list = ['1631049.818'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Beveik pastovus nedidelis EMG 
list = ['1630996.837'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Silpnas BW per visą įrašą, apie 118500 išlenda BW šuolis kartu su EMG
list = ['1637580.246'] # 1,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Labai važinėjanti izolinija, kartais staigiai, raumenų triukšmų palyginti nedaug.
list = ['1630996.212'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0

# Po pertraukos
# Švarus
list = ['1637629.641'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Pirmoje įrašo pusėje nemažai BW, taip pat ir staigių, yra EMG ir EMA epizodų
list = ['1630983.707'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Silpnas, bet pastovus BW
list = ['1633936.216'] # 1,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# EMA, EMG, yra staigių BW
list = ['1630999.968'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Silpnas, bet pastovus BW
list = ['1631085.276'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 2, pridėtų: 1
# Silpnas BW, kartais pasirodo nedidelis EMG
list = ['1638804.264'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Yra BW su šuoliais, epizodai su nedideliu EMG
list = ['1637184.858'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 2, pridėtų: 1
# Antroje įrašo pusėje atsirado didesnio BW epizodai, pirma pusė - švari
list = ['1630984.960'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Silpnas, bet pastovus BW (išskyrus epizodą ties 100000)
list = ['1633936.844'] # 1,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Nemažai stiprių EMG, yra EMA, daug BW
list = ['1630808.859'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 1, pridėtų: 2
# Švarus
# 
# Po pertraukos 
list = ['1625405.717'] # 2,  nesutampančių 3, anotatoriaus panaikintų rpikų: 2, pridėtų: 1
# 
list = ['1643077.573'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Pastovus, bet nedidelis BW (išskyrus fragmentą ties 97500), pasitaiko epizodai su nestipriu EMG
list = ['1630761.032'] # 2,  nesutampančių 3, anotatoriaus panaikintų rpikų: 0, pridėtų: 3
# 
list = ['1630810.723'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 0, pridėtų: 3
# 
list = ['1636501.893'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Nedidelis, bet nuolatinis BW
list = ['1633583.663'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 2, pridėtų: 1
# Labai gera kokybė, išskyrus BW kalnelius ties 54000 ir 115000
list = ['1632727.496'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 0, pridėtų: 3
# Gera kokybė su vietomis nedideliu EMG, tik vienas triukšminis epizodas ties 98400
list = ['1630808.238'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 0, pridėtų: 3
# Gera kokybė
list = ['1632724.351'] # 1,  nesutampančių 3, anotatoriaus panaikintų rpikų: 0, pridėtų: 3
# Prasta pirmos dalie kokybė: BW, EMG, EMA
list = ['1634737.213'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Nedidelis, bet dažnas BW, kartais pasirodo nedidelis EMG
list = ['1643068.627'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 2, pridėtų: 1
# Nedidelis EMG per visą įrašą, keletas ryškių BW epizodų

# 2023 05 10
list = ['1637094.317'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# BW bangos ir kalneliai, kartais pasirodo ir ryškesnis EMG
list = ['1631734.441'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Labai švarus
list = ['1643069.905'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Nedidelis BW ir labai silpnas EMG
list = ['1630717.159'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 2, pridėtų: 1
# 
list = ['1630727.721'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Didžioji dalis švari, tik pasitaiko staigių BW ir keletas EMA
list = ['1644866.492'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 1, pridėtų: 2
# nuolatinis BW, bet stipresnis tik keletoje vietų 
list = ['1636411.382'] # 0,  nesutampančių 3, anotatoriaus panaikintų rpikų: 2, pridėtų: 1
# Labai švarus
list = ['1635144.614'] # 1,  nesutampančių 3, anotatoriaus panaikintų rpikų: 3, pridėtų: 0
# Antroje dalyje pasitaiko nemažai EMA
list = ['1638912.435'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus
list = ['1638916.776'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus
list = ['1638914.295'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus
list = ['1631013.491'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus, labai silpnas EMG

# 2023 05 11
list = ['1638915.536'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Labai švarus
list = ['1640633.548'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 0, pridėtų: 2
# Švarus, tik poroje vietų neaišku - lyg tai triukšmas, lyg tai S
list = ['1637619.776'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Nemažai BW kalniukų, silpnas EMG
list = ['1646777.284'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Gana švarus, išskyrus vieną šuolį, nedidelis BW 
list = ['1633430.440'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Nedidelis BW, bet dažnas, yra šuolis
list = ['1639464.936'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus, išskyrus trumpą epizodą su EMG
list = ['1638796.812'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Pirma pusė gana švari, antroje pusėje išlenda epizodai su EMA (ties 78600 ir 94000)
list = ['1631048.565'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Kartai pasirodo nestiprus EMG
list = ['1637189.186'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Nedidelis, bet pastovus BW, ties 119000 pasirodo didelė BW banga
list = ['1638797.432'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Nuolatinis BW, bet stipresnis tik keletoje vietų, vietomis nedidelis EMG
list = ['1638798.056'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Nuolatinis BW, bet stipresnis tik keletoje vietų, vietomis nedidelis EMG

# Po pertraukos
list = ['1625400.796'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Daug BH, yra epizodų su EMG
list = ['1638805.509'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Švarus, tik pora smulkių trikdžių
list = ['1630812.584'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Švarus
list = ['1630987.458'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Nedidelis BW
list = ['1630811.963'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 0, pridėtų: 2
# Nedidelis BW ir kartais nedidelis EMG
list = ['1630809.483'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 0, pridėtų: 2
# Švarus
list = ['1633395.812'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Yra keletas epizodų su staigiu BW ir nedideliu EMG
list = ['1633424.804'] # 2,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Antroje dalyje šokinėjantis BW, epizodai su EMG, EMA
list = ['1636501.269'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Nedidelis, bet nuolatinis BW
list = ['1638814.549'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus, išskyrus epizodą EMA plius EMG ties 110250
list = ['1626924.927'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# izolinija šokinėja nedaug, vietomis išlenda raumenų triušmai
list = ['1631079.860'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus, išskyrus pora trumpų epizodų su staigiu BW
list = ['1633526.601'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Švarus, labai nedidelis BW ir trumpi epizodai nedidelio EMG

# 2023 05 12
list = ['1630806.377'] # 1,  nesutampančių 2, anotatoriaus panaikintų rpikų: 0 pridėtų: 2
# Neigiamą vektorių turinčios ekstrasistolės. Atkreipti dėmesį testuojant R dantelius
#  atpažįstantį algoritmą
# Yra BW bangų ir šuolių, EMG maži, kartais matosi EMA epizodų
list = ['1626933.710'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 0, pridėtų: 2
# BW nedidelis, bet nuolatinis, yra ryškių EMG epizodų
list = ['1633410.244'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Nedidelis BW ir EMG, bet dažni
list = ['1633323.647'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Labai triukšminga pirma pusė: BW nedidelis, bet dažnai pasirodo EMG ir EMA
list = ['1636423.305'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Švarus, tik keletas EMG epizodų
list = ['1636503.768'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus, tik nedidelis BW per visą įrašą
list = ['1636486.195'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Švarus
list = ['1634734.705'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# BW nedidelės bangos, pora staigesnių šuolių, nedidelis, bet dažnas EMG
list = ['1634734.078'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# BW bangos ir šuoliai
list = ['1634730.943'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# BW bangos ir šuoliai, kartais pasirodo EMG


# Po pertraukos
list = ['1631075.483'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Labai švarus
list = ['1632726.869'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 0, pridėtų: 2
# Tik viena BW kalnelis ties 122000, nuolat pasirodo silpnas  EMG
list = ['1643065.525'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Nuolatinis silpnas EMG
list = ['1643663.847'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Nuolatinis nedidelis BW, poroje vietų BW šuoliai su EMG pliūpsniais,  yra EMS epizodas
list = ['1640019.242'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Švarus, nedidelis BW
list = ['1638813.922'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Yra II° AV blokados epizodas. Švarus
list = ['1637023.085'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Nuolatinis nedidelis BW, gana švarus, išskyrus trikdžius ties 115750
list = ['1636480.010'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Pirma pusė švari, antroje pusėje prasideda nedidelis BW ir vietomis trikdžiai

# 2023 05 13
list = ['1631075.931'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Pirma dalis švari, antroje dalyje yra epizodų su BW, EMG ir EMA
list = ['1630813.935'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Yra keletas epizodų su BW, EMG, EMA
list = ['1633540.147'] # 1,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Didžiojoje įrašo dalyje kokybė gera, kartais išlenda nedideli raumenų triukšmai,
# tačiau pabaigoje išlenda nemaži raumenų triukšmai
list = ['1644334.517'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Pasitaiko epizodų su EMG, taip pat yra keletas epizodų su BW šuoliais
list = ['1638828.999'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Nuolatinis nedidelis BW (išskyrus vieną šuolį)
list = ['1637262.757'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Nuolatinis nedidelis BW, epizodai su EMG, ties 92000 epizodas su didesniais trikdžiais
list = ['1646775.410'] # 1,  nesutampančių 2, anotatoriaus panaikintų rpikų: 1, pridėtų: 1
# Labai nestabilus BW
list = ['1630979.952'] # 0,  nesutampančių 2, anotatoriaus panaikintų rpikų: 2, pridėtų: 0
# Nedidelis, bet nuolatinis BW
list = ['1638909.955'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus
list = ['1638911.815'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus
list = ['1643054.465'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus
list = ['1638918.640'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus
list = ['1643053.840'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus
list = ['1638914.916'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus


# po pertraukos
list = ['1630939.246'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus, nedidelis BW
list = ['1638204.161'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis BW ir EMG
list = ['1639456.175'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis EMG, yra keleta epizodų su BW šuoliais, tikėtina EMA
list = ['1630994.958'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# BW, bet staigių nėra, vienas didesnis kalnelis
list = ['1630938.621'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis BW, išskyrus vieną kitą kalnelį, antroje dalyje pradeda matytis EMG
list = ['1630937.367'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Yra keletas epizodų su ryškiu EMG ir BW kalneliu, šiaip BW nedidelis
list = ['1642460.045'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis, pastovus BW, nedaug trikdžių
list = ['1630971.181'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Pastovus nedidelis BW, silpnas EMG, kartais sustiprėja
list = ['1638795.571'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nuolatinis nedidelis BW, kartais pereinantis į šuolius. Yra nestiprus EMG, kartais pasirodo EMA
list = ['1630759.790'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 0, pridėtų: 1
# EMG, kartais nesilpnas, pastovus, netvarkingas  BW  
list = ['1630937.996'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis BW, nedidelis EMG, bet yra keleta staigių BW šuolių su padidėjusiu EMG
list = ['1638794.951'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Yra visko: banguojančio ir šokinėjančio BW, taip pat EMG ir EMA
list = ['1639459.928'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus

# 2023 05 xx
list = ['1638804.885'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 0, pridėtų: 1
# Nežymus BW, labai silpnas EGM, bet yra trys epizodai su BW staigiu šuoliu
list = ['1630973.062'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Šiaip gera kokybė, tik įrašo pradžioje yra keletas didelio EMG, ir BW šuolio epizodų
list = ['1630968.051'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Šiaip gera kokybė, tik įrašo pabaigoje  yra epizodas su ryškiu  EMG ir staigiu BW šuoliu
list = ['1630993.079'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Šiaip gera kokybė, yra tik pora epizodų su padidėjusiu  EMG ir staigiu BW šuoliu
list = ['1633379.501'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus, tik vienas epizodas su BW šuoliu
list = ['1630758.549'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 0, pridėtų: 1
# Nedidelis BW, ryškesnis įrašo pabaigoje
list = ['1643076.946'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis BW ir kartais - EMG
list = ['1625403.256'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Antroje dalyje yra nemažai BW bangavimų ir epizodų su EMG, gal net EMA

# Po pertraukos
list = ['1633386.401'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Labai intensyvus BW
list = ['1633345.610'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus, poroje vietų - nedidelis EMG
list = ['1630811.343'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 0, pridėtų: 1
# Nedidelis BW ir kartais nedidelis EMG
list = ['1633368.199'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Didžiulių šuolių nėra (išskyrus vieną), bet BW dažnas, išlenda ir EMG
list = ['1633361.296'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis BW, retsykiais pasirodo EMG
list = ['1633352.508'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Vietomis nedidelis BW, keleta epizodų su EMG
list = ['1633328.044'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Vietomis nedidelis BW, keleta epizodų su EMG
list = ['1633406.480'] # 2,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Yra gana didelės amplitudės raumenų triukšmų
list = ['1633346.864'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis BW su keliais BW šuoliais,  karts nuo karto išlenda EMG
list = ['1633331.805'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nemažai ilgų epizodų su staigiais BW, EMG, EMA
list = ['1630982.455'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus, labai nedidelis nuolatinis BW
list = ['1626330.744'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Yra S. Nedidelis, bet pastovus  BW, yra epizodų su EMG
list = ['1633342.470'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus, tik pabaigoje pasirodo BW ir EMG. Bendras komentaras visiems 
# įrašams: pacientui ypač būdingos interpoliuotos skivelinės ekstrasistolės.
list = ['1634732.824'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis BW
list = ['1630673.825'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 0, pridėtų: 1
# NERADAU SĄRAŠE !!!
list = ['1638815.807'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# II° AV blokados epizodas. Įrašas švarus


# 2023 05 15
list = ['1633373.219'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Trumpi nedidelio BW epizodai
list = ['1633369.457'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis BW, pabaigoje pasirodo nedidelis EMG
list = ['1633338.709'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nemažai BW bangelių, pabaigoje pasirodo silpnas EMG
list = ['1633509.207'] # 2,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Daug EMA, ypač pirmoje pusėje
list = ['1630777.178'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 0, pridėtų: 1
# Trikdis pasirodo pačioje pabaigoje
list = ['1633382.636'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nemažai EMG epizodų, yra BW šuolių
list = ['1634727.173'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Yra visko: BW, EMG, EMA
list = ['1630736.382'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Labai švarus
list = ['1638815.176'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Gana švarus, išskyrus epizodą ties 1750 ir 116000 
list = ['1630983.083'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis nuolatinis BW, labai silpnas EMG 

# po pertraukos
list = ['1633043.190'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis, bet nuolatinis BW, kartais pasirodo labai nedidelis EMG, trikdis ties  60800
list = ['1638218.882'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Gana švarus, išskyrus epizodą ties 82250 ir pabaigą 
list = ['1638821.479'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# aug BW, nors ir nedidelio, daug EMA, ypač antroje dalyje
list = ['1643069.252'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Per visą įrašą matosi silpnas EMG, yra pora epizodų su BW šuoliu
list = ['1638825.866'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Su pastoviu nedideliu BW
list = ['1630725.247'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus, išskyrus keturius trumpus epizodus su trikdžiais
list = ['1640034.194'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus, išskyrus trumpus  trikdžius poroje vietų
list = ['1630986.835'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Yra BW bangų, ypač pirmoje dalyje, kartais matosi silpnas EMG
list = ['1644286.178'] # 1,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Yra EMA epizodų
list = ['1630974.942'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Nedidelis nuolatinis BW, labai silpnas EMG 
list = ['1638808.902'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Švarus, tik visą laiką matosi labai silpnas EMG , o poroje vietų jis pasidaro ryškus
list = ['1630947.382'] # 2,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Prastas įrašas, pilnas visokių trikdžių
list = ['1633031.843'] # 2,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Būtų geras įrašas, tačiau pabaigoje keletas netrumpų epizodų su įvairiausiais trikdžiais
list = ['1642011.715'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# BW yra, bet nedidelis, išskyrus keletą šuolių pradžioje
list = ['1630808.306'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# Yra keletas epizodų su BW šuoliu
list = ['1630808.306'] # 0,  nesutampančių 1, anotatoriaus panaikintų rpikų: 1, pridėtų: 0
# BW be didelių šuolių, keleta epizodų su padidėjusių EMG

# PABAIGA


flag_eval_copy = True 

for file_name in list:
    # file_name = str(file_name)    
    print(f"\nZive įrašas:  {file_name:>2}")

    if (flag_eval_copy):
        # Perrašome failus į vertinamų failų aplanką
        src = Path(rec_dir, file_name)
        dst = Path(rec_dir_eval, file_name)

        # kopijuojame duomenis ir .json failus, išsaugodami metaduomenis su copy2
        shutil.copy2(src, dst)
        src = Path(rec_dir, file_name + '.json')
        dst = Path(rec_dir_eval, file_name + '.json')
        shutil.copy2(src, dst)
        print(f"\nFailai {file_name} ir {file_name+ '.json'} perrašyti  iš {rec_dir} į {rec_dir_eval}")
        print()

        # I-a dalis: nuskaitome rpeaks iš json failo

    filepath = Path(rec_dir, file_name + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    rpeaks_dict = data['rpeaks']
    rpeaks_from_json = np.array([dict['sampleIndex'] for dict in rpeaks_dict])
    # rpeaks_from_json = np.array([3,5,7,10,11,12,13,15,17,19,20,22,23,24])
    # print(rpeaks_from_json)

    print(f"rpeaks iš json failo: {len(rpeaks_from_json)}")
    # print(rpeaks_json) 

    # Suformuojame supaprastintą json failą sulyginimui, prie vardo pridėta 'js'
    # Sulyginimui naudoju http://www.jsondiff.com/
    # data = [{"sampleIndex":int(c), "annotationValue":"N"} for c in rpeaks_from_json]
    # filename = file_name + '-js' + '.json'
    # with open(filename, 'w', encoding='UTF-8') as outfile:
    #     json.dump(data, outfile)
    # print(data)    


        # II-a dalis: suformuojame rpeaks su Neurokitu,
        # pakartojant Zive skriptą iš analysis.py ir heartrate_analysis.py
    filepath = Path(rec_dir, file_name)
    signal_raw = zive_read_file_1ch(filepath)
    # signal = nk.signal_filter(signal=signal_raw, sampling_rate=200, lowcut=0.5, method="butterworth", order=5)
    signal = signal_raw    
    ecg_signal_df = pd.DataFrame(signal, columns=['orig'])

    analysis_results = AnalyseHeartrate(ecg_signal_df)
    rpeaks_from_signal = analysis_results['rpeaks']
    print(f"rpeaks iš signal: {len(rpeaks_from_signal)}")

    # print(rpeaks_from_signal)
    
  # Suformuojame supaprastintą json failą sulyginimui, prie vardo pridėta 'sg'
    # data = [{"sampleIndex":int(c), "annotationValue":"N"} for c in rpeaks_from_signal]
    # filename = file_name + '-sg' + '.json'
    # with open(filename, 'w', encoding='UTF-8') as outfile:
    #     json.dump(data, outfile)
    # print(data)    


            # Sulyginimas

    ab = np.setdiff1d(rpeaks_from_json, rpeaks_from_signal)
    print("\nViso rankiniu būdu pridėtų rpikų:", len(ab))
    print("Reikšmės faile rpeaks_from_json kurių nėra faile rpeaks_from_signal")
    # https://numpy.org/doc/stable/reference/generated/numpy.setdiff1d.html
    # print(ab)   # Return the unique values in a that are not in b
    flag = np.size(ab)
    if flag:
        sorter = np.argsort(rpeaks_from_json)
        idxs = sorter[np.searchsorted(rpeaks_from_json, ab, sorter=sorter)]
        print(f"i     index   rpeak")
        for i in range(len(idxs)):
            print(f"{i} {idxs[i]:>9} {ab[i]:>7}")
        # print(idxs)
    else:
        print('Tokių reikšmių nėra')

    ba = np.setdiff1d(rpeaks_from_signal,rpeaks_from_json)
    print("\nViso rankiniu būdu panaikintų rpikų:", len(ba))
    print("Reikšmės faile rpeaks_from_signal kurių nėra faile rpeaks_from_json")
    # print(ba)   # Return the unique values in b that are not in a
    flag = np.size(ba)
    if flag:
        sorter = np.argsort(rpeaks_from_signal)
        idxs = sorter[np.searchsorted(rpeaks_from_signal, ba, sorter=sorter)]
        print(f"i     index   rpeak")
        for i in range(len(idxs)):
            print(f"{i} {idxs[i]:>9} {ba[i]:>7}")
        # print(idxs)
    else:
        print('Tokių reikšmių nėra')





OS in my system :  linux
Skriptas rpeaks sulyginimui. Lyginamas rpeaks masyvas, suformuotas iš json failo,
nuskaityto iš Zive frontendo ir rpeaks masyvas, gautas iš EKG signalo naudojant
Neurokit2 funkciją ecg_peaks. EKG signalas (įrašas) irgi gautas iš Zive frontendo.
Abu rpeaks masyvai įrašomi į json vizualiniam sulyginimui su http://www.jsondiff.com/
rpeaks iš json failo varde pridėta '-js', rpeaks iš signalo - pridėta '-sg'

Zive įrašas:  1630808.306

Failai 1630808.306 ir 1630808.306.json perrašyti  iš /home/kesju/DI/DUOM_2022_RUDUO_2/records_selected į /home/kesju/DI/DUOM_2022_RUDUO_2/records_evaluated

rpeaks iš json failo: 637
rpeaks iš signal: 638

Viso rankiniu būdu pridėtų rpikų: 0
Reikšmės faile rpeaks_from_json kurių nėra faile rpeaks_from_signal
Tokių reikšmių nėra

Viso rankiniu būdu panaikintų rpikų: 1
Reikšmės faile rpeaks_from_signal kurių nėra faile rpeaks_from_json
i     index   rpeak
0       546  109086


In [26]:
for file_name in list:
    file_name = str(file_name)    
    print(f"\nZive įrašas:  {file_name:>2}")

        # I-a dalis: nuskaitome rpeaks iš json failo

    filepath = Path(rec_dir, file_name + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    rpeaks_dict = data['rpeaks']
    rpeaks_from_json = np.array([dict['sampleIndex'] for dict in rpeaks_dict])
    # print(rpeaks_from_json[:5])

            # II-a dalis: suformuojame rpeaks su Neurokitu

    filepath = Path(rec_dir, file_name)
    signal_raw = zive_read_file_1ch(filepath)
    _, rpeaks = nk.ecg_peaks(signal_raw, sampling_rate=200, correct_artifacts=False)
    rpeaks_from_signal = rpeaks['ECG_R_Peaks']
    # print(rpeaks_from_signal[:5])
    
  
            # Sulyginimas

    ab = np.setdiff1d(rpeaks_from_json, rpeaks_from_signal)
    ba = np.setdiff1d(rpeaks_from_signal,rpeaks_from_json)
    print(f"Nesutampančių rpeaks skaičius NS: {np.size(ab)+np.size(ba)} ")





Zive įrašas:  1630808.306
Nesutampančių rpeaks skaičius NS: 1 
